In [1]:
import tensorflow as tf
import numpy as np
from qutip import *
import itertools as it
import tomography

In [2]:
N = 2 

In [3]:
trainsize = 1000

In [4]:
testsize = 100

In [5]:
def experiment(phi,m=100,exact=True):
    #initialization
    res = []
    s = [qeye(2), sigmax(), sigmay(), sigmaz()]
    olist = []
    for i in it.product(range(4), repeat=N):
        temp = []
        for j in i:
            temp.append(s[j])
        tempop = tensor(temp)
        olist.append(tempop)
    if exact:
        res = [expect(o,phi) for o in olist]
    
    else:
        for o in olist:
            vals, vecs = o.eigenstates()
            ps = expect(ket2dm(phi),vecs)
            ps = ps
            out = np.random.choice(vals, m, p=ps)
            res.append(np.mean(out))
    
    return res 

def gen_entangled():
    phi0 = 1/np.sqrt(2)*(tensor([basis(2,0) for dummy in range(N)]) + tensor([basis(2,1) for dummy in range(N)]))
    U = tensor([rand_unitary(2) for dummy in range(N)])
    return U*phi0 

def gen_prod():
    phi0 = tensor([basis(2,0) for dummy in range(N)])
    U = tensor([rand_unitary(2) for dummy in range(N)])
    return U*phi0


almost any random state is entangled, so I just generate random kets for entangled training set

In [6]:
psilisttrain = []
for j in range(trainsize):
    psitemp = rand_ket(2**N)
    psitemp.dims = [[2,2],1]
    psilisttrain.append(psitemp)

In [7]:
ent = [experiment(psi1) for psi1 in psilisttrain]

In [8]:
prod = [experiment(gen_prod()) for dummy in range(trainsize)]

In [9]:
traindatax = np.concatenate((ent,prod))

In [10]:
traindatay = np.concatenate(([[0,1] for dummy in range(trainsize)],[[1,0] for dummy in range(trainsize)]))

Shuffle the trainin so that batches are randomized 

In [13]:
enttest = [experiment(gen_entangled()) for dummy in range(testsize)]
prodtest = [experiment(gen_prod()) for dummy in range(testsize)]
testdatax = np.concatenate((enttest,prodtest))
testdatay = np.concatenate(([[0,1] for dummy in range(testsize)],[[1,0] for dummy in range(testsize)]))

In [14]:
randomizer = np.random.permutation(2*trainsize)

In [15]:
np.take(traindatax,randomizer,axis=0,out=traindatax);
np.take(traindatay,randomizer,axis=0,out=traindatay);

Setting up the neural network and initializing it

In [17]:
x = tf.placeholder(tf.float32, [None, 4**N])

In [18]:
W = tf.Variable(tf.zeros([4**N, 2]))
b = tf.Variable(tf.zeros([2]))

In [19]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [20]:
y_ = tf.placeholder(tf.float32, [None, 2])

In [21]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [22]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [23]:
sess = tf.InteractiveSession()

In [25]:
tf.global_variables_initializer().run()

This is the actual training step

In [26]:
for _ in range(20):
    batch_xs = traindatax[50*_:50*(_+1)]
    batch_ys = traindatay[50*_:50*(_+1)]
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

Defining error

In [27]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [28]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [30]:
print(sess.run(accuracy, feed_dict={x: testdatax, y_: testdatay}))

0.955


Individual predictions

In [37]:
psi = rand_ket(2**N)
psi.dims = [[2,2],1]

In [38]:
entropy_vn(ket2dm(psi).ptrace(0))

0.5182686810393877

In [39]:
testest = experiment(psi,10)

In [40]:
predict = tf.argmax(y, 1)
pred = predict.eval({x: [testest] })
pred

array([1])